In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset = pd.read_csv('../input/avocado-prices/avocado.csv')
dataset.head(10)

In [ ]:
dataset.info()

In [ ]:
nan = dataset.isnull().sum()
nan

In [ ]:
ds_sample = dataset.sample(100)
ds_sample

In [ ]:
# hangi kolonda kaç tane null var?

ds_isnull = dataset.isnull().sum()
ds_isnull


In [ ]:
df_desc = dataset.describe() 

In [ ]:
# korelasyon matrisi

corrmat = dataset.corr()
corrmat

In [ ]:
# korelasyon matrisinin görselleştirilmesi

plt.subplots(figsize=(15,12))
sns.heatmap(corrmat, vmax=1, cmap="Blues", square=True)

In [ ]:
#"AvocadoPrice" ve diğer  özelliklerinin korelasyon matrisinin oluşturulması

averagePrice = dataset["AveragePrice"]
corr_averagePrice = dataset.corrwith(averagePrice, axis=0)
corr_averagePrice = pd.DataFrame(corr_averagePrice)
corr_averagePrice.rename (columns = {0: 'AveragePrice'}, inplace = True)

#Oluşturulan korelasyon matrisinin görselleştirilmesi
plt.subplots(figsize=(15,12))
sns.set(font_scale=1.1)
sns.heatmap(corr_averagePrice, vmax=1, cmap="Blues",annot=True)

**DATA CLEAN**

In [ ]:
df = dataset.drop(columns = ["Unnamed: 0"])
df

In [ ]:
#%% total bags =? small + large + xlarge

df_bags = df[["Total Bags", "Small Bags", "Large Bags", "XLarge Bags"]]
df_bags["total2"] = df_bags["Small Bags"] + df_bags["Large Bags"] + df_bags["XLarge Bags"]
df_bags["diff"] = df_bags["Total Bags"] - df_bags["total2"]
df_bags

In [ ]:
df = df.drop(columns = ["Total Bags"])

In [ ]:
regions = df["region"].unique()
regions

In [ ]:
df["date2"] = pd.to_datetime(df["Date"])
df["month"] = pd.DatetimeIndex(df["date2"]).month
df = df.drop(columns=["Date"])
df = df.drop(columns=["date2"])

In [ ]:
df

In [ ]:
#"AvocadoPrice" ve diğer  özelliklerinin korelasyon matrisinin oluşturulması

averagePrice = df["AveragePrice"]
corr_averagePrice = df.corrwith(averagePrice, axis=0)
corr_averagePrice = pd.DataFrame(corr_averagePrice)
corr_averagePrice.rename (columns = {0: 'AveragePrice'}, inplace = True)

#Oluşturulan korelasyon matrisinin görselleştirilmesi
plt.subplots(figsize=(15,12))
sns.set(font_scale=1.1)
sns.heatmap(corr_averagePrice, vmax=1, cmap="Blues",annot=True)

In [ ]:
df_dummy = pd.get_dummies(df[["type", "region"]])


df = df.drop(columns=["type"])
df = df.drop(columns=["region"])

df = pd.concat([df,df_dummy], axis=1)

**MODEL**

In [ ]:
y = df["AveragePrice"]
X = df.drop(columns = ["AveragePrice"])

X, y = shuffle(X, y, random_state = 7)

In [ ]:
#  70% of the data will be used for train
#  30% will be used for testing

row_size = df.shape[0]  # int 506
offset = int(row_size * 0.7) # int 354.2

# X_train = X[:] overfit olur 
X_train = X[:offset]  # Generating X_train data
X_test = X[offset:]   # Generating X_test data


y_train = y[:offset]  # Generating y_train data
y_test = y[offset:]  # yGenerating y_test data

In [ ]:
model = RandomForestRegressor()

tuned = model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred

In [ ]:
# score
score_r2 = metrics.r2_score(y_test, y_pred)
score_r2

In [ ]:
rf_Importance = pd.DataFrame({"Importance":tuned.feature_importances_*100},index = X_train.columns)

s = rf_Importance.sort_values(by = "Importance", axis=0, ascending = False)
fig_dims = (50, 50)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "Importance", y =  rf_Importance["Importance"].index, ax=ax, data=s);

In [ ]:
new_train_columns = []

for index, row in rf_Importance.iterrows():
    if(row["Importance"] >= 0.06):
        new_train_columns.append(index)
    
new_train = X_train[new_train_columns]

X_train = new_train
X_test = X_test[new_train_columns]

model = RandomForestRegressor()

tuned = model.fit(X_train, y_train)

y_pred = model.predict(X_test )
y_pred

In [ ]:
score_r2 = metrics.r2_score(y_test, y_pred)
score_r2